In [1]:
import parcels
print(parcels.__version__)

2.1.6.dev11+g38e73cf


In [1]:
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
from datetime import timedelta as delta
from glob import glob
import numpy as np
import xarray as xr
import os
import warnings
warnings.simplefilter('ignore', category=xr.SerializationWarning)

In [2]:
# to find the indices of a specific lon-lat box
def getclosest_ij(lats,lons,latpt,lonpt):    
    """Function to find the index of the closest point to a certain lon/lat value."""
    dist_sq = (lats-latpt)**2 + (lons-lonpt)**2                 # find squared distance of every point on grid
    minindex_flattened = dist_sq.argmin()                       # 1D index of minimum dist_sq element
    return np.unravel_index(minindex_flattened, lats.shape)     # Get 2D index for latvals and lonvals arrays from 1D index

initialgrid_mask = '/data/oceanparcels/input_data/NEMO-MEDUSA/ORCA0083-N006/means/ORCA0083-N06_20070105d05U.nc'
mask = xr.open_dataset(initialgrid_mask, decode_times=False)
Lat, Lon, Depth = mask.variables['nav_lat'], mask.variables['nav_lon'], mask.variables['depthu']
latvals = Lat[:]; lonvals = Lon[:] # extract lat/lon values to numpy arrays

iy_min, ix_min = getclosest_ij(latvals, lonvals, -7, -120)
iy_max, ix_max = getclosest_ij(latvals, lonvals, 7, -70)

print(ix_min)

In [ ]:
ddir = "/data/oceanparcels/input_data/NEMO-MEDUSA/ORCA0083-N006/"
#odir = "../../results/output_data"
#fname = os.path.join(odir,"galapagosparticles_testrun.nc")
fname = "galapagosparticles_testrun.nc"

# set field, only year 2000
ufiles = sorted(glob(ddir+'means/ORCA0083-N06_200[0-0]*d05U.nc'))
vfiles = [u.replace('05U.nc', '05V.nc') for u in ufiles]
meshfile = glob(ddir+'domain/coordinates.nc')
nemofiles = {'U': {'lon': meshfile, 'lat': meshfile, 'data': ufiles},
             'V': {'lon': meshfile, 'lat': meshfile, 'data': vfiles}}
nemovariables = {'U': 'uo', 'V': 'vo'}
nemodimensions = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}
fieldset = FieldSet.from_nemo(nemofiles, nemovariables, nemodimensions, field_chunksize='auto')

fU = fieldset.U
fieldset.computeTimeChunk(fU.grid.time[0], 1)    #forward
#fieldset.computeTimeChunk(fU.grid.time[-1], -1)  #backward

In [ ]:
#initialize where to start particles
galapagos_extent = [-91.8, -89, -1.4, 0.7]
startlon, startlat = np.meshgrid(np.arange(galapagos_extent[0], galapagos_extent[1], 0.2),
                                 np.arange(galapagos_extent[2], galapagos_extent[3], 0.2))

In [ ]:
#functions to add to the kernel
def Age(fieldset, particle, time):
    particle.age = particle.age + math.fabs(particle.dt)
    if particle.age > 30*86400:
        particle.delete()

def WrapParticle(particle, fieldset, time):
    if particle.lon < -530:
        particle.lon += 360        

#additional features of the particles        
class GalapagosParticle(JITParticle):
    age = Variable('age', initial = 0.)            

In [ ]:
# set particle conditions
pset = ParticleSet(fieldset=fieldset,
                   pclass=GalapagosParticle,
                   lon=startlon,
                   lat=startlat,
                   time=fU.grid.time[0])

In [ ]:
outfile = pset.ParticleFile(name=fname, outputdt=delta(days=1))

pset.execute(AdvectionRK4+pset.Kernel(Age)+WrapParticle,
             dt=delta(hours=1),
             output_file=outfile)

outfile.export()
outfile.close()